In [ ]:

from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

import time

# 強制等待 (執行期間休息一下)
from time import sleep

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文
my_options.add_experimental_option(
    "prefs", 
    {
        "profile.managed_default_content_settings.images": 2,  # 2 = 禁用圖片
        "profile.default_content_setting_values.images": 2
    }
)

url = "https://www.ruten.com.tw/category/00210001/list?prc.now=1000-"

def check_element_exists():
    """檢查目標元素是否存在"""
    try:
        element = driver.find_element(By.CLASS_NAME, "search-result-container.bottom-part")
        return True
    except NoSuchElementException:
        return False

def scroll_and_check():
    """捲到最底部並檢查元素"""
    print("開始捲動到頁面底部...")
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        # 捲到底部
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 等待內容載入
        
        # 檢查元素是否存在
        if check_element_exists():
            print("找到目標元素！")
            return True
        
        # 計算新的高度
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        # 如果高度沒變化，表示到底部了
        if new_height == last_height:
            print("已到達頁面底部")
            return check_element_exists()
            
        last_height = new_height
        print("繼續捲動...")

def get_products_info():
    """
    抓取商品名稱和價格的方法，分別從頂部和底部容器獲取
    
    Returns:
        list: 包含所有商品資訊的列表，每個商品是一個字典
    """
    products_data = []
    
    # 找到頂部和底部的容器
    containers = [
        "search-result-container.top-part",
        "search-result-container.bottom-part"
    ]
    
    for container_class in containers:
        try:
            # 找到容器
            container = driver.find_element(By.CLASS_NAME, container_class)
            # 在容器中找到所有商品項目
            product_items = container.find_elements(By.CLASS_NAME, "product-item")
            
            for item in product_items:
                try:
                    # 抓取商品名稱
                    name = item.find_element(By.CLASS_NAME, "rt-product-card-name").text
                    
                    # 抓取價格（可能有一個或兩個價格）
                    prices = item.find_elements(By.CLASS_NAME, "rt-text-price.rt-text-bold.text-price-dollar")
                    price_texts = [price.text for price in prices]
                    
                    # 將資訊加入列表
                    products_data.append({
                        "名稱": name,
                        "價格": price_texts,
                        "來源": "頂部" if container_class.endswith("top-part") else "底部"
                    })
                    
                except Exception as e:
                    print(f"處理商品時發生錯誤: {str(e)}")
                    continue
                    
        except Exception as e:
            print(f"處理容器 {container_class} 時發生錯誤: {str(e)}")
            continue
    
    return products_data

driver = webdriver.Chrome(
    options = my_options,
)
driver.get(url)

try:
    success = scroll_and_check()
    if success:
        print("成功找到目標元素")
        
        # 獲取商品資訊
        products = get_products_info()
        
        # 顯示結果
        print(f"\n總共找到 {len(products)} 個商品")
        for i, product in enumerate(products, 1):
            print(f"\n商品 {i}:")
            print(f"名稱: {product['名稱']}")
            if len(product['價格']) == 1:
                print(f"價格: {product['價格'][0]}")
            else:
                print(f"價格範圍: {' ~ '.join(product['價格'])}")
    else:
        print("未能找到目標元素")

except Exception as e:
    print(f"執行時發生錯誤: {str(e)}")
    
finally:
    # 可選：完成後關閉瀏覽器
    driver.quit()
    pass
